In [ ]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Flatten, Dense, Dropout, AveragePooling2D
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(f"tf.__version__ = {tf.__version__}")

In [ ]:
DATA_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
DATA_BASE_DIR = './data'
INPUT_SIZE = 224
NUM_CLASSES = 5
BATCH_SIZE = 32

In [ ]:
os.makedirs(DATA_BASE_DIR, exist_ok=True)

data_dir = tf.keras.utils.get_file(
    origin=DATA_URL, 
    fname=os.path.splitext(os.path.basename(DATA_URL))[0], 
    cache_dir=DATA_BASE_DIR, 
    cache_subdir='', 
    untar=True
)

In [ ]:
train_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.1,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

train_iter = train_generator.flow_from_directory(
    data_dir, 
    subset='training',
    seed=123,
    target_size=(INPUT_SIZE, INPUT_SIZE), 
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
)

val_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.1,
)

val_iter = val_generator.flow_from_directory(
    data_dir, 
    subset='validation',
    seed=123,
    target_size=(INPUT_SIZE, INPUT_SIZE), 
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
)

In [ ]:
base_model = MobileNetV2(
    alpha=1.4,
    weights='imagenet',
    include_top=False,
    pooling=None,
    input_shape=(INPUT_SIZE, INPUT_SIZE, 3),
    classes=NUM_CLASSES
)

In [ ]:
# for layer in base_model.layers:
#     layer.trainable = False

In [ ]:
model = Sequential()
model.add(base_model)

model.add(AveragePooling2D(pool_size=(7, 7)))
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(
    loss=CategoricalCrossentropy(label_smoothing=0.1), 
    optimizer=Adam(lr=1e-4), 
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
DECAY_START = 1

def scheduler(epoch):
    if epoch < DECAY_START:
        return 5e-5
    else:
        return max(1e-4 * np.exp(0.3 * (DECAY_START - epoch)), 1e-7)

lr_scheduler = LearningRateScheduler(scheduler, verbose=1)

In [ ]:
history = model.fit(
    train_iter,
    steps_per_epoch=len(train_iter),
    validation_data=val_iter,
    validation_steps=len(val_iter),
    epochs=20,
    callbacks=[lr_scheduler],
    verbose=1
)

In [ ]:
tf.saved_model.save(model, './saved_model')

In [ ]:
import utils

df_result = utils.evaluate(model, val_iter)
df_wrong = df_result.loc[df_result['y'] != df_result['y_hat'], ]
utils.plot_images(df_wrong)